In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *

In [ ]:
class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)


In [ ]:
class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()


In [ ]:
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Emission Graph No-Code Solution')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.add_circle_btn.move(0, 0)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)

    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()


### With Legend 

In [ ]:
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Emission Graph No-Code Solution')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.add_circle_btn.move(0, 0)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)

        # Add legend items
        self.legend_activity = QLabel('Blue arrow represents Activity Level', self)
        self.legend_activity.move(10, 80)
        self.scene.addWidget(self.legend_activity)

        self.legend_emission = QLabel('Red disk represents Emission Source', self)
        self.legend_emission.move(10, 100)
        self.scene.addWidget(self.legend_emission)

    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()


### With Heatmap

In [ ]:
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Emission Graph No-Code Solution')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.add_circle_btn.move(0, 0)
        self.scene.addWidget(self.add_circle_btn)

        self.heatmap_btn = QPushButton('Show Heatmap')
        self.heatmap_btn.clicked.connect(self.show_heatmap)
        self.heatmap_btn.move(0, 30)
        self.scene.addWidget(self.heatmap_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)

    def show_heatmap(self):
        heatmap = QGraphicsRectItem(0, 0, 500, 500)
        gradient = QLinearGradient(0, 0, 0, 500)
        gradient.setColorAt(0.0, Qt.yellow)
        gradient.setColorAt(0.5, Qt.red)
        gradient.setColorAt(1.0, Qt.black)
        heatmap.setBrush(QBrush(gradient))
        heatmap.setOpacity(0.7)
        self.scene.addItem(heatmap)

        for circle in self.scene.items():
            if isinstance(circle, Circle):
                for arrow in self.scene.items():
                    if isinstance(arrow, Arrow) and arrow.endItem == circle:
                        val = circle.value * arrow.value
                        x, y = circle.pos().x() + circle.boundingRect().width() / 2, circle.pos().y() + circle.boundingRect().height() / 2
                        r = val * 50
                        ellipse = QGraphicsEllipseItem(x - r, y - r, 2 * r, 2 * r)
                        ellipse.setBrush(QColor(255, 255, 255, 100))
                        self.scene.addItem(ellipse)


### Execute

In [ ]:
app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())

In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()
        
        self.heatmap_layer = None
        
        self.heatmap_widget = None

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()
            self.calculate_sum()
            self.update_text_position()
            self.update_heatmap()
            
    def update_heatmap(self):
        if self.heatmap_widget:
            self.scene.removeItem(self.heatmap_widget)
            self.heatmap_layer = None
        for item in self.scene.items():
            if isinstance(item, App) and item.add_heatmap_btn.isChecked():
                heatmap = calculate_heatmap(self.scene)
                if self.heatmap_layer is None:
                    self.heatmap_layer = item.heatmap_ax.imshow(heatmap, cmap='Reds', alpha=0.5)
                else:
                    self.heatmap_layer.set_data(heatmap)
                    self.heatmap_layer.set_extent([xmin, xmax, ymin, ymax])
                item.heatmap_canvas.draw()
                self.heatmap_widget = item.heatmap_canvas

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)

class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()
            
def calculate_heatmap(scene):
    # Get all the circles in the scene
    circles = [item for item in scene.items() if isinstance(item, Circle)]

    # Get the x, y, and value for each circle
    xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
    ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
    values = [circle.value for circle in circles]

    # Calculate the product between the circle values and arrow values
    heatmap_values = np.zeros((len(circles), len(circles)))
    for i, start_circle in enumerate(circles):
        for j, end_arrow in enumerate(scene.items()):
            if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                k = circles.index(end_arrow.startItem)
                heatmap_values[i][k] += end_arrow.value * start_circle.value

    # Create the heatmap figure
    fig = Figure(figsize=(5, 5), dpi=100)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    im = ax.imshow(heatmap_values, cmap='plasma')
    fig.colorbar(im)

    # Set the tick labels to the circle numbers
    ax.set_xticks(range(len(circles)))
    ax.set_yticks(range(len(circles)))
    ax.set_xticklabels([str(circle.num) for circle in circles])
    ax.set_yticklabels([str(circle.num) for circle in circles])
    ax.tick_params(axis='both', labelsize=8)

    return canvas
            
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Emission Graph No-Code Solution')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.add_circle_btn.move(0, 0)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)
        
        self.heatmap_canvas = FigureCanvas(Figure())
        self.heatmap_ax = self.heatmap_canvas.figure.subplots()
        
        self.add_heatmap_btn = QPushButton('Add Heatmap')
        self.add_heatmap_btn.clicked.connect(self.add_heatmap)
        self.add_heatmap_btn.move(0, 30)
        self.scene.addWidget(self.add_heatmap_btn)
        


    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')
        
    def add_heatmap(self):
        heatmap = calculate_heatmap(self.scene)
        print(heatmap)
        print(type(heatmap))
        self.heatmap_ax.imshow(np.asarray(heatmap), cmap='Reds', alpha=0.5)
        heatmap_widget = self.scene.addWidget(heatmap)
        heatmap_widget.setZValue(2)

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()
        
    def calculate_heatmap(scene):
        # Get all the circles in the scene
        circles = [item for item in scene.items() if isinstance(item, Circle)]

        # Get the x, y, and value for each circle
        xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
        ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
        values = [circle.value for circle in circles]

        # Calculate the product between the circle values and arrow values
        heatmap_values = np.zeros((len(circles), len(circles)))
        for i, start_circle in enumerate(circles):
            for j, end_arrow in enumerate(scene.items()):
                if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                    k = circles.index(end_arrow.startItem)
                    heatmap_values[i][k] += end_arrow.value * start_circle.value

        # Create the heatmap figure
        fig = Figure(figsize=(5, 5), dpi=100)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        im = ax.imshow(heatmap_values, cmap='plasma')
        fig.colorbar(im)

        # Set the tick labels to the circle numbers
        ax.set_xticks(range(len(circles)))
        ax.set_yticks(range(len(circles)))
        ax.set_xticklabels([str(circle.num) for circle in circles])
        ax.set_yticklabels([str(circle.num) for circle in circles])
        ax.tick_params(axis='both', labelsize=8)

        return canvas
    


app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())

In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()
        
        self.heatmap_widget = None

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()
            self.calculate_sum()
            self.update_text_position()
            self.update_heatmap()
            
    def update_heatmap(self):
        if self.heatmap_widget:
            self.scene.removeItem(self.heatmap_widget)
            self.heatmap_widget = None
        for item in self.scene.items():
            if isinstance(item, App) and item.add_heatmap_btn.isChecked():
                self.heatmap_widget = item.add_heatmap()

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)

class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(-1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()
        self.setZValue(-2)

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()
            
def calculate_heatmap(scene):
    # Get all the circles in the scene
    circles = [item for item in scene.items() if isinstance(item, Circle)]

    # Get the x, y, and value for each circle
    xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
    ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
    values = [circle.value for circle in circles]

    # Calculate the product between the circle values and arrow values
    heatmap_values = np.zeros((len(circles), len(circles)))
    for i, start_circle in enumerate(circles):
        for j, end_arrow in enumerate(scene.items()):
            if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                k = circles.index(end_arrow.startItem)
                heatmap_values[i][k] += end_arrow.value * start_circle.value

    # Create the heatmap figure
    fig = Figure(figsize=(5, 5), dpi=100)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    im = ax.imshow(heatmap_values, cmap='plasma')
    fig.colorbar(im)

    # Set the tick labels to the circle numbers
    ax.set_xticks(range(len(circles)))
    ax.set_yticks(range(len(circles)))
    ax.set_xticklabels([str(circle.num) for circle in circles])
    ax.set_yticklabels([str(circle.num) for circle in circles])
    ax.tick_params(axis='both', labelsize=8)

    return canvas
            
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Emission Graph No-Code Solution')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.move(20, 40)
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)
        
        self.add_heatmap_btn = QPushButton('Add Heatmap')
        self.add_heatmap_btn.move(10, 70)
        self.add_heatmap_btn.clicked.connect(self.add_heatmap)
        self.scene.addWidget(self.add_heatmap_btn)
        


    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')
        
    def add_heatmap(self):
        heatmap = calculate_heatmap(self.scene)
        heatmap_widget = self.scene.addWidget(heatmap)
        heatmap_widget.setZValue(1)

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()
        
    def calculate_heatmap(scene):
        # Get all the circles in the scene
        circles = [item for item in scene.items() if isinstance(item, Circle)]

        # Get the x, y, and value for each circle
        xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
        ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
        values = [circle.value for circle in circles]

        # Calculate the product between the circle values and arrow values
        heatmap_values = np.zeros((len(circles), len(circles)))
        for i, start_circle in enumerate(circles):
            for j, end_arrow in enumerate(scene.items()):
                if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                    k = circles.index(end_arrow.startItem)
                    heatmap_values[i][k] += end_arrow.value * start_circle.value

        # Create the heatmap figure
        fig = Figure(figsize=(5, 5), dpi=100)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        #ax.set_position([0.1,0.1])
        im = ax.imshow(heatmap_values, cmap='plasma')
        fig.colorbar(im)

        # Set the tick labels to the circle numbers
        ax.set_xticks(range(len(circles)))
        ax.set_yticks(range(len(circles)))
        ax.set_xticklabels([str(circle.num) for circle in circles])
        ax.set_yticklabels([str(circle.num) for circle in circles])
        ax.tick_params(axis='both', labelsize=8)

        return canvas
    


app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())

In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5.QtWidgets import *

class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()
        
        self.heatmap_widget = None

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()
            self.calculate_sum()
            self.update_text_position()
            self.update_heatmap()
            self.update()

    def update_heatmap(self):
        if self.heatmap_widget:
            self.scene.removeItem(self.heatmap_widget)
            self.heatmap_widget = None
        for item in self.scene.items():
            if isinstance(item, App) and item.add_heatmap_btn.isChecked():
                self.heatmap_widget = item.add_heatmap()

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)

    def paint(self, painter, option, widget=None):
        super().paint(painter, option, widget)
        if self.value == 0:
            painter.setBrush(QColor(255, 255, 255))
            painter.drawEllipse(self.boundingRect().center(), self.boundingRect().width() / 2.5, self.boundingRect().height() / 2.5)
class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(-1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()
        self.setZValue(-2)

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()
            
def calculate_heatmap(scene):
    # Get all the circles in the scene
    circles = [item for item in scene.items() if isinstance(item, Circle)]

    # Get the x, y, and value for each circle
    xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
    ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
    values = [circle.value for circle in circles]

    # Calculate the product between the circle values and arrow values
    heatmap_values = np.zeros((len(circles), len(circles)))
    for i, start_circle in enumerate(circles):
        for j, end_arrow in enumerate(scene.items()):
            if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                k = circles.index(end_arrow.startItem)
                heatmap_values[i][k] += end_arrow.value * start_circle.value

    # Create the heatmap figure
    fig = Figure(figsize=(5, 5), dpi=100)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    im = ax.imshow(heatmap_values, cmap='plasma')
    fig.colorbar(im)

    # Set the tick labels to the circle numbers
    ax.set_xticks(range(len(circles)))
    ax.set_yticks(range(len(circles)))
    ax.set_xticklabels([str(circle.num) for circle in circles])
    ax.set_yticklabels([str(circle.num) for circle in circles])
    ax.tick_params(axis='both', labelsize=8)

    return canvas
            
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Carbon Accounting - Emission Graph')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.move(200, 700)
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)
        
        self.add_heatmap_btn = QPushButton('Add Heatmap')
        self.add_heatmap_btn.move(10, 700)
        self.add_heatmap_btn.clicked.connect(self.add_heatmap)
        self.scene.addWidget(self.add_heatmap_btn)
        
        # Add reset button
        reset_button = QPushButton("Reset")
        reset_button.clicked.connect(self.reset)
        top_layout.addWidget(reset_button, alignment=Qt.AlignRight)
        
    def reset(self):
        for item in self.scene.items():
            if isinstance(item, Circle) or isinstance(item, Arrow):
                self.scene.removeItem(item)
        self.sum_label.setText("Total Emission: 0")
        self.scene.update()

    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')
        
    def add_heatmap(self):
        heatmap = calculate_heatmap(self.scene)
        heatmap_widget = self.scene.addWidget(heatmap)
        heatmap_widget.setZValue(1)

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()
        
    def calculate_heatmap(scene):
        # Get all the circles in the scene
        circles = [item for item in scene.items() if isinstance(item, Circle)]

        # Get the x, y, and value for each circle
        xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
        ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
        values = [circle.value for circle in circles]

        # Calculate the product between the circle values and arrow values
        heatmap_values = np.zeros((len(circles), len(circles)))
        for i, start_circle in enumerate(circles):
            for j, end_arrow in enumerate(scene.items()):
                if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                    k = circles.index(end_arrow.startItem)
                    heatmap_values[i][k] += end_arrow.value * start_circle.value

        # Create the heatmap figure
        fig = Figure(figsize=(5, 5), dpi=100)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        #ax.set_position([0.1,0.1])
        im = ax.imshow(heatmap_values, cmap='plasma')
        fig.colorbar(im)

        # Set the tick labels to the circle numbers
        ax.set_xticks(range(len(circles)))
        ax.set_yticks(range(len(circles)))
        ax.set_xticklabels([str(circle.num) for circle in circles])
        ax.set_yticklabels([str(circle.num) for circle in circles])
        ax.tick_params(axis='both', labelsize=8)

        return canvas
    


app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()
        
        self.heatmap_widget = None

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()
            self.calculate_sum()
            self.update_text_position()
            self.update_heatmap()
            self.update()

    def update_heatmap(self):
        if self.heatmap_widget:
            self.scene.removeItem(self.heatmap_widget)
            self.heatmap_widget = None
        for item in self.scene.items():
            if isinstance(item, App) and item.add_heatmap_btn.isChecked():
                self.heatmap_widget = item.add_heatmap()

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)

    def paint(self, painter, option, widget=None):
        super().paint(painter, option, widget)
        if self.value == 0:
            painter.setBrush(QColor(255, 255, 255))
            painter.drawEllipse(self.boundingRect().center(), self.boundingRect().width() / 2.5, self.boundingRect().height() / 2.5)
class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(-1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()
        self.setZValue(-2)

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()
            
def calculate_heatmap(scene):
    # Get all the circles in the scene
    circles = [item for item in scene.items() if isinstance(item, Circle)]

    # Get the x, y, and value for each circle
    xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
    ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
    values = [circle.value for circle in circles]

    # Calculate the product between the circle values and arrow values
    heatmap_values = np.zeros((len(circles), len(circles)))
    for i, start_circle in enumerate(circles):
        for j, end_arrow in enumerate(scene.items()):
            if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                k = circles.index(end_arrow.startItem)
                heatmap_values[i][k] += end_arrow.value * start_circle.value

    # Create the heatmap figure
    fig = Figure(figsize=(5, 5), dpi=100)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    im = ax.imshow(heatmap_values, cmap='plasma')
    fig.colorbar(im)

    # Set the tick labels to the circle numbers
    ax.set_xticks(range(len(circles)))
    ax.set_yticks(range(len(circles)))
    ax.set_xticklabels([str(circle.num) for circle in circles])
    ax.set_yticklabels([str(circle.num) for circle in circles])
    ax.tick_params(axis='both', labelsize=8)

    return canvas
            
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Carbon Accounting - Emission Graph')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.move(200, 700)
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)
        
        self.add_heatmap_btn = QPushButton('Add Heatmap')
        self.add_heatmap_btn.move(10, 700)
        self.add_heatmap_btn.clicked.connect(self.add_heatmap)
        self.scene.addWidget(self.add_heatmap_btn)
        


    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')
        
    def add_heatmap(self):
        heatmap = calculate_heatmap(self.scene)
        heatmap_widget = self.scene.addWidget(heatmap)
        heatmap_widget.setZValue(1)

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()
        
    def calculate_heatmap(scene):
        # Get all the circles in the scene
        circles = [item for item in scene.items() if isinstance(item, Circle)]

        # Get the x, y, and value for each circle
        xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
        ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
        values = [circle.value for circle in circles]

        # Calculate the product between the circle values and arrow values
        heatmap_values = np.zeros((len(circles), len(circles)))
        for i, start_circle in enumerate(circles):
            for j, end_arrow in enumerate(scene.items()):
                if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                    k = circles.index(end_arrow.startItem)
                    heatmap_values[i][k] += end_arrow.value * start_circle.value

        # Create the heatmap figure
        fig = Figure(figsize=(5, 5), dpi=100)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        #ax.set_position([0.1,0.1])
        im = ax.imshow(heatmap_values, cmap='plasma')
        fig.colorbar(im)

        # Set the tick labels to the circle numbers
        ax.set_xticks(range(len(circles)))
        ax.set_yticks(range(len(circles)))
        ax.set_xticklabels([str(circle.num) for circle in circles])
        ax.set_yticklabels([str(circle.num) for circle in circles])
        ax.tick_params(axis='both', labelsize=8)

        return canvas
    


app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())



AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

...

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

AttributeError: 'QGraphicsScene' object has no attribute 'sum_label'

SystemExit: 0

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

class Circle(QGraphicsEllipseItem):
    def __init__(self, x, y, r, num, scene):
        super().__init__(x - r, y - r, 2 * r, 2 * r)
        self.num = num
        self.scene = scene
        self.setBrush(QColor(255, 0, 0))
        self.setFlag(QGraphicsEllipseItem.ItemIsMovable)
        self.setFlag(QGraphicsEllipseItem.ItemSendsGeometryChanges)
        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setDefaultTextColor(QColor(255, 255, 255))
        self.text.setFont(QFont("Arial", 10))
        self.update_text_position()
        
        self.heatmap_widget = None

    def itemChange(self, change, value):
        if change == QGraphicsEllipseItem.ItemPositionHasChanged:
            for arrow in self.scene.items():
                if isinstance(arrow, Arrow):
                    arrow.updatePosition()
            self.calculate_sum()
            self.update_text_position()
        return super().itemChange(change, value)

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the Emission Factor:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.text.setPlainText(str(self.value))
            self.calculate_sum()
            self.update_text_position()
            self.calculate_sum()
            self.update_text_position()
            self.update_heatmap()
            self.update()

    def update_heatmap(self):
        if self.heatmap_widget:
            self.scene.removeItem(self.heatmap_widget)
            self.heatmap_widget = None
        for item in self.scene.items():
            if isinstance(item, App) and item.add_heatmap_btn.isChecked():
                self.heatmap_widget = item.add_heatmap()

    def calculate_sum(self):
        total_sum = 0
        for arrow in self.scene.items():
            if isinstance(arrow, Arrow) and arrow.endItem == self:
                total_sum += arrow.value * self.value
        sum_text = "Total Emission: " + str(total_sum)
        self.scene.sum_label.setText(sum_text)

    def update_text_position(self):
        rect = self.boundingRect()
        text_rect = self.text.boundingRect()
        x = rect.center().x() - text_rect.center().x()
        y = rect.center().y() - text_rect.center().y()
        self.text.setPos(x, y)

    def paint(self, painter, option, widget=None):
        super().paint(painter, option, widget)
        if self.value == 0:
            painter.setBrush(QColor(255, 255, 255))
            painter.drawEllipse(self.boundingRect().center(), self.boundingRect().width() / 2.5, self.boundingRect().height() / 2.5)
class Arrow(QGraphicsLineItem):
    def __init__(self, startItem, endItem):
        super().__init__()
        self.startItem = startItem
        self.endItem = endItem
        self.setPen(QPen(QColor(0, 0, 255), 2, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
        self.setFlag(QGraphicsLineItem.ItemIsSelectable)

        self.arrow_head = QGraphicsPolygonItem(self)
        self.arrow_head.setPen(QPen(Qt.NoPen))
        self.arrow_head.setBrush(QBrush(QColor(0, 0, 255)))
        self.arrow_head.setZValue(-1)

        self.value = 0
        self.text = QGraphicsTextItem(str(self.value), self)
        self.text.setPos(self.midpoint() + QPointF(-7, -18))
        self.text.setDefaultTextColor(QColor(0, 0, 255))
        self.text.setFont(QFont("Arial", 10))

        self.updatePosition()
        self.setZValue(-2)

    def midpoint(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        return QPointF((start.x() + end.x()) / 2, (start.y() + end.y()) / 2)

    def updatePosition(self):
        start = self.startItem.scenePos() + QPointF(self.startItem.rect().center())
        end = self.endItem.scenePos() + QPointF(self.endItem.rect().center())
        line = QLineF(start, end)

        self.setLine(line)

        angle = line.angle()

        head_points = [QPointF(0, 0), QPointF(-10, -10), QPointF(10, -10)]

        head_polygon = QPolygonF(head_points)
        head_transform = QTransform()
        head_transform.translate(line.p2().x(), line.p2().y())
        head_transform.rotate(angle)
        self.arrow_head.setPolygon(head_transform.map(head_polygon))

        self.text.setPlainText(str(self.value))
        self.text.setPos(self.midpoint() + QPointF(-7, -18))

    def mouseDoubleClickEvent(self, event):
        value, ok = QInputDialog.getDouble(None, "Set Value", "Enter a value for the activity level:", self.value, -9999999, 9999999, 2)
        if ok:
            self.value = value
            self.updatePosition()
            
def calculate_heatmap(scene):
    # Get all the circles in the scene
    circles = [item for item in scene.items() if isinstance(item, Circle)]

    # Get the x, y, and value for each circle
    xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
    ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
    values = [circle.value for circle in circles]

    # Calculate the product between the circle values and arrow values
    heatmap_values = np.zeros((len(circles), len(circles)))
    for i, start_circle in enumerate(circles):
        for j, end_arrow in enumerate(scene.items()):
            if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                k = circles.index(end_arrow.startItem)
                heatmap_values[i][k] += end_arrow.value * start_circle.value

    # Create the heatmap figure
    fig = Figure(figsize=(5, 5), dpi=100)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    im = ax.imshow(heatmap_values, cmap='plasma')
    fig.colorbar(im)

    # Set the tick labels to the circle numbers
    ax.set_xticks(range(len(circles)))
    ax.set_yticks(range(len(circles)))
    ax.set_xticklabels([str(circle.num) for circle in circles])
    ax.set_yticklabels([str(circle.num) for circle in circles])
    ax.tick_params(axis='both', labelsize=8)

    return canvas
            
class App(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Carbon Accounting - Emission Graph')
        self.scene = QGraphicsScene(self)
        self.setScene(self.scene)
        self.setRenderHint(QPainter.Antialiasing, True)

        self.num_circles = 0
        self.prev_circle = None

        self.add_circle_btn = QPushButton('Add Emission Source')
        self.add_circle_btn.move(200, 700)
        self.add_circle_btn.clicked.connect(self.add_circle)
        self.scene.addWidget(self.add_circle_btn)

        self.sum_label = QLabel('Total Emission: 0', self)
        self.sum_label.move(10, 10)
        self.sum_label.setAlignment(Qt.AlignTop)
        self.sum_label.setMinimumWidth(10000)
        self.sum_label.setMaximumWidth(10000)

        self.update_sum_btn = QPushButton('Update', self)
        self.update_sum_btn.move(10, 50)
        self.update_sum_btn.clicked.connect(self.calculate_sum)
        
        self.add_heatmap_btn = QPushButton('Add Heatmap')
        self.add_heatmap_btn.move(10, 700)
        self.add_heatmap_btn.clicked.connect(self.add_heatmap)
        self.scene.addWidget(self.add_heatmap_btn)
        


    def calculate_sum(self):
        total_sum = 0
        for item in self.scene.items():
            if isinstance(item, Arrow):
                start_circle = item.startItem
                end_circle = item.endItem
                arrow_value = item.value
                start_value = start_circle.value
                end_value = end_circle.value
                product = arrow_value * end_value
                total_sum += product
        self.sum_label.setText(f'Total Emissions: {total_sum} T CO2')
        
    def add_heatmap(self):
        heatmap = calculate_heatmap(self.scene)
        heatmap_widget = self.scene.addWidget(heatmap)
        heatmap_widget.setZValue(1)

    def add_circle(self):
        self.num_circles += 1
        circle = Circle(50*self.num_circles, 50, 20, self.num_circles, self.scene)
        self.scene.addItem(circle)
        if self.prev_circle is not None:
            arrow = Arrow(self.prev_circle, circle)
            self.scene.addItem(arrow)
        self.prev_circle = circle
        self.calculate_sum()
        
    def calculate_heatmap(scene):
        # Get all the circles in the scene
        circles = [item for item in scene.items() if isinstance(item, Circle)]

        # Get the x, y, and value for each circle
        xs = [circle.x() + circle.rect().width() / 2 for circle in circles]
        ys = [circle.y() + circle.rect().height() / 2 for circle in circles]
        values = [circle.value for circle in circles]

        # Calculate the product between the circle values and arrow values
        heatmap_values = np.zeros((len(circles), len(circles)))
        for i, start_circle in enumerate(circles):
            for j, end_arrow in enumerate(scene.items()):
                if isinstance(end_arrow, Arrow) and end_arrow.endItem == start_circle:
                    k = circles.index(end_arrow.startItem)
                    heatmap_values[i][k] += end_arrow.value * start_circle.value

        # Create the heatmap figure
        fig = Figure(figsize=(5, 5), dpi=100)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        #ax.set_position([0.1,0.1])
        im = ax.imshow(heatmap_values, cmap='plasma')
        fig.colorbar(im)

        # Set the tick labels to the circle numbers
        ax.set_xticks(range(len(circles)))
        ax.set_yticks(range(len(circles)))
        ax.set_xticklabels([str(circle.num) for circle in circles])
        ax.set_yticklabels([str(circle.num) for circle in circles])
        ax.tick_params(axis='both', labelsize=8)

        return canvas
    


app = QApplication(sys.argv)
ex = App()
ex.show()
sys.exit(app.exec_())

